# FUNCIONES

LIBRERIAS Y AUTENTIFICACIONES

In [2]:
# importamos las librerías que necesitaremos:

# Uso de API's
# -----------------------------------------------------------------------
import requests

# Importar librería para interactuar con la API de Spotify y generar los credenciales necesarios para la autentificación 
# -----------------------------------------------------------------------
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Importar librería para trabajar con variables de entorno en Python, permite cargar variables de entorno desde un archivo .env en el directorio del proyecto
# -----------------------------------------------------------------------
import os
from dotenv import load_dotenv

# Importar librería para trabajar con archivos JSON
# -----------------------------------------------------------------------
import json

# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
# Importar librerías para procesamiento de texto
# -----------------------------------------------------------------------
import re


# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames


In [3]:
load_dotenv()

True

In [4]:
sp_clave = os.getenv("Spotify_id")
sp_contraseña = os.getenv("Spotify_secret")

In [17]:
####celda maria borrar#####
sp_clave = os.getenv("client_id")
sp_contraseña = os.getenv("client_secret")

In [20]:
########celda maria borrar2 ######
import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=sp_clave,
                                               client_secret=sp_contraseña,
                                               redirect_uri="https://www.google.com",
                                               scope="user-library-read"))

In [18]:
#Autentificación SPOTIFY:
auth_manager = sp = SpotifyClientCredentials(client_id=sp_clave,
                                             client_secret=sp_contraseña)

sp = spotipy.Spotify(auth_manager=auth_manager)

In [6]:
#Autentificación LASTFM:
lastfm_clave = os.getenv("lastfm_id")

FUNCIÓN SACAR CANCIONES DE SPOTIFY

SERIA GUAY HACER UNA LISTA CON TODOS LOS GENEROS PARA QUE SAQUE LA FUNCION UN CSV YA COMBINADO

In [31]:
canciones = {"nombre_artista": [], "nombre_canción": [], "año_lanzamiento": [], "género_musical": [], "tipo": []}

def canciones_spotify(género):
    for anio in range(2019,2024):
        for offset in range(0, 250, 50):
            datos_trance = sp.search(q=f"genre:{género}, year:{anio}", type="track", limit=50, offset=offset)
        # valorar si podemos meter el género como un range o enumerate o algo así para meter los 4 géneros
            for cancion in datos_trance["tracks"]["items"]:
                canciones["nombre_artista"].append(cancion["album"]["artists"][0]["name"])
                canciones["nombre_canción"].append(cancion["name"])
                canciones["género_musical"].append("trance")
                canciones["año_lanzamiento"].append(cancion["album"]["release_date"][0:4]) 
                canciones["tipo"].append(cancion["type"])

    df_spotify = pd.DataFrame(canciones)

    return df_spotify # y conseguir un df de los 4 géneros


#genero= trance , techno , pop, rock

FUNCION PARA SACAR TODOS LOS GENEROS AL MISMO TIEMPO

In [ ]:
def canciones_spotifycombinada():
    canciones = {"nombre_artista": [], "nombre_canción": [], "año_lanzamiento": [], "género_musical": [], "tipo": []}
    generos = ['pop','rock','techno','trance']
    for anio in range(2019,2024):
        for offset in range(0, 250, 50):
            for genero in generos:
                datos_trance = sp.search(q=f"genre:{genero}, year:{anio}", type="track", limit=50, offset=offset)
                # valorar si podemos meter el género como un range o enumerate o algo así para meter los 4 géneros
                for cancion in datos_trance["tracks"]["items"]:
                    canciones["nombre_artista"].append(cancion["album"]["artists"][0]["name"])
                    canciones["nombre_canción"].append(cancion["name"])
                    canciones["género_musical"].append(f"{genero}")
                    canciones["año_lanzamiento"].append(cancion["album"]["release_date"][0:4])
                    canciones["tipo"].append(cancion["type"])
    df_spotify = pd.DataFrame(canciones)
    return df_spotify # y conseguir un df de los 4 géneros

In [ ]:
canciones_spotify("trance")

In [ ]:
canciones_spotifycombinada() #para llamar a la función de la base de datos combinada

,nombre_artista,nombre_canción,año_lanzamiento,género_musical,tipo
0,BTS,HOME,2019,pop,track
1,Ariana Grande,"thank u, next",2019,pop,track
2,Sebastian Yatra,Un Año,2019,pop,track
3,Alejandro Sanz,Mi Persona Favorita,2019,pop,track
4,Lana Del Rey,Doin' Time,2019,pop,track
...,...,...,...,...,...
4931,Various Artists,Sonic Adventure,2023,techno,track
4932,Fred Linger,Time Out,2023,techno,track
4933,Various Artists,The Sun,2023,techno,track
4934,Various Artists,Shot On Top - Deetech Remix,2023,techno,track


In [ ]:
#####llamar a spotify por la función , sino solo con el df_spotify peta#####
canciones_spotify("trance").to_csv("spotifydatabase2222.csv")

In [ ]:
#esto estaría fallando al tener la función creada
df_spotify.to_csv("spotifydatabase.csv")

NameError: name 'df_spotify' is not defined

In [ ]:
df_values_spotify=pd.read_csv("spotifydatabase.csv",index_col=0)
# ponemos el index_col= 0 para q no nos cree el Unnamed

In [ ]:
df_values_spotify
# el índice va a salir pero lo quitamos al hacer el itertuples

In [ ]:
list(df_values_spotify.itertuples(index=False, name=None))
# aqui no incluye ni el índice ni los títulos de las colmnas

values_spotify = list(df_values_spotify.itertuples(index=False, name=None))

FUNCIÓN SACAR CANCIONES DE LASTFM

In [ ]:
pd.read_csv("spotifydatabase.csv", index_col=0)

In [ ]:
cosaslista = pd.read_csv("spotifydatabase.csv")
columnanombreartista = cosaslista["nombre_artista"].tolist()

In [ ]:
def llamar_api(url):
    llamada = requests.get(url)
    
    if llamada.status_code != 200:
        print(llamada.reason)
    else:
        return llamada.json()

In [ ]:
infoartistas={"nombre_artista":[],"bio":[],"listeners":[],"playcount":[],"similar":[]}

for nombreartista in columnanombreartista:
    
    if nombreartista == "Various Artists":  # elimiamos Various artists para q no salgan datos erróneos en las bio
        continue

    elif nombreartista in infoartistas["nombre_artista"]: 
        continue  # Skip this iteration if the artist is a duplicate 

    else:    
        try: 
            url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={nombreartista}&api_key={lastfm_clave}&format=json"
            respuesta = llamar_api(url)

            #Para hacer esto: infoartistas["similar"].append(respuesta["artist"]["similar"]["artist"][0]['name'])
            similares=[]
            for artistaejemplo in respuesta["artist"]["similar"]["artist"]:
                similares.append(artistaejemplo['name'])

            infoartistas["similar"].append(similares)
            infoartistas["nombre_artista"].append(nombreartista)
            infoartistas["bio"].append(respuesta["artist"]["bio"]["summary"])
            infoartistas["listeners"].append(respuesta["artist"]["stats"]["listeners"])
            infoartistas["playcount"].append(respuesta["artist"]["stats"]["playcount"])

            

        except KeyError:
            continue 

print(infoartistas)

In [ ]:
df=pd.DataFrame(infoartistas)

In [ ]:
df.to_csv("lastfmdatabase.csv")

In [ ]:
df_values_lastfm=pd.read_csv("lastfmdatabase.csv",index_col=0)

In [ ]:
values_lastfm = list(df_values_lastfm.itertuples(index=False, name=None))
# aqui no incluye ni el índice ni los títulos de las colmnas

FUNCIONES DE BBDD MUSIC TRENDS

- FUNCIÓN CREAR BASE DE DATOS MUSICTRENDS:

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1')

mycursor = cnx.cursor()
sql = "CREATE DATABASE MusicTrends"


try: 
    mycursor.execute(sql)
    print(mycursor.rowcount, "base de datos creada.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

- FUNCIÓN CREAR TABLA LASTFM:

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='MusicTrends')


mycursor = cnx.cursor()
sql = "CREATE TABLE LastFM (id_artista INT AUTO_INCREMENT NOT NULL, nombre_artista VARCHAR(100) NOT NULL, bio LONGTEXT NULL, listeners INT NULL, playcount INT NULL, similar LONGTEXT NULL, PRIMARY KEY (id_artista)"

try: 
    mycursor.execute(sql)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

- FUNCIÓN CREAR TABLA SPOTIFY:

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='MusicTrends')


mycursor = cnx.cursor()
sql = "CREATE TABLE Spotify (id_track INT NOT NULL AUTO_INCREMENT, nombre_canción VARCHAR(200) NULL, nombre_artista VARCHAR(100) NOT NULL, id_artista INT NULL, año_lanzamiento VARCHAR(4), género_musical VARCHAR(45) NULL, tipo VARCHAR(10) NULL, PRIMARY KEY(id_track), CONSTRAINT fk_LastFM FOREIGN KEY (id_artista) REFERENCES LastFM(id_artista) ON DELETE CASCADE ON UPDATE CASCADE)"

try: 
    mycursor.execute(sql)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

- FUNCIÓN INSERTAR DATOS EN LASTFM:

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='MusicTrends')


mycursor = cnx.cursor()
sql = "INSERT INTO LastFM (nombre_artista, bio, listeners, playcount, similar) VALUES (%s, %s, %s, %s,%s)"
val = values_lastfm

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

- FUNCIÓN DE CREAR DICCIONARIO DE ARTISTAS:

In [41]:
##########celda maria test borrar si eso##########

def dictartistas():
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                                host='127.0.0.1', database='prueba_musictrends')


    mycursor = cnx.cursor()
    mycursor.execute("SELECT id_artista, nombre_artista FROM LastFM")
    artistas = mycursor.fetchall()
    # Diccionario: nombre_artista.lower() -> id_artista para relacionar el id en las dos tablas
    diccionario_artistas = {nombre.lower(): id for id, nombre in artistas}


    diccionario_artistas


    valores_spotify = []

    for fila in valores_spotify:
            nombre_artista = fila[0].lower()
            id_artista = diccionario_artistas.get(nombre_artista)

            if id_artista is not None:
                # Creamos: (id_artista, nombre_artista, nombre_cancion, año_lanzamiento, género_musical, tipo)
                valores_spotify.append((id_artista, *fila))
            else:
                valores_spotify.append((None, *fila))
                print(f"⚠️ Artista no encontrado en LastFM: {nombre_artista}")


    valores_spotify # ver si tiene sentido con este nombre

In [42]:
dictartistas()

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='MusicTrends')


mycursor = cnx.cursor()
mycursor.execute("SELECT id_artista, nombre_artista FROM LastFM")
artistas = mycursor.fetchall()
# Diccionario: nombre_artista.lower() -> id_artista para relacionar el id en las dos tablas
diccionario_artistas = {nombre.lower(): id for id, nombre in artistas}


diccionario_artistas

In [ ]:
valores_spotify = []

for fila in values_spotify:
    nombre_artista = fila[0].lower()
    id_artista = diccionario_artistas.get(nombre_artista)

    if id_artista is not None:
        # Creamos: (id_artista, nombre_artista, nombre_cancion, año_lanzamiento, género_musical, tipo)
        valores_spotify.append((id_artista, *fila))
    else:
        valores_spotify.append((None, *fila))
        print(f"⚠️ Artista no encontrado en LastFM: {nombre_artista}")


valores_spotify # ver si tiene sentido con este nombre

- FUNCIÓN INSERTAR DATOS EN SPOTIFY:

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='MusicTrends')


mycursor = cnx.cursor()
sql = "INSERT INTO Spotify (id_artista, nombre_artista, nombre_canción, año_lanzamiento, género_musical, tipo) VALUES (%s, %s, %s, %s,%s,%s)"
val = values_spotify

try: 
    mycursor.executemany(sql, valores_spotify)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)